In [ ]:
import pandas as pd

# Load the raw insurance data
insurance = pd.read_csv('/home/emine2/ATMS-402/colsom_1989_2024.csv')
insurance

/tmp/ipykernel_2085893/3635813231.py:4: DtypeWarning: Columns (8,9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  insurance = pd.read_csv('/home/emine2/ATMS-402/colsom_1989_2024.csv')


In [ ]:
# Filter rows where commodity_year_identifier == 2024, commodity_name is "corn", and state_abbreviation is "IL"
filtered = insurance[
    (insurance["commodity_year_identifier"] == 2022) &
    (insurance["commodity_name"].str.lower() == "corn") &
    (insurance["state_abbreviation"] == "IA")
]
filtered.head()

In [7]:
# 1) Inspect the raw data
print("--- Initial Data Inspection ---")
print("Raw data shape:", insurance.shape)
print("Original data types:")
print(insurance[['commodity_year_identifier', 'state_code', 'commodity_code', 'cause_of_loss_code']].dtypes)

# 2) Clean Data and Enforce Data Types
# We will convert all code/ID columns to numeric types.
# Using errors='coerce' is a robust way to handle this. It will turn any value
# that cannot be converted into a number into `NaN` (Not a Number).
# These NaN rows will be automatically excluded by the filters later on.
print("\n--- Cleaning and Converting Data Types ---")

# List of columns that should be integers
integer_columns = [
    'commodity_year_identifier',
    'state_code',
    'commodity_code',
    'cause_of_loss_code'
]

for col in integer_columns:
    insurance[col] = pd.to_numeric(insurance[col], errors='coerce')

# Drop rows where our key identifiers became NaN after conversion, and convert to standard integer
insurance.dropna(subset=integer_columns, inplace=True)
insurance[integer_columns] = insurance[integer_columns].astype(int)

print("Data types after conversion:")
print(insurance[['commodity_year_identifier', 'state_code', 'commodity_code', 'cause_of_loss_code']].dtypes)
print("Shape after cleaning:", insurance.shape)


# 3) Apply string transformations
insurance["commodity_name"] = insurance["commodity_name"].str.lower()
insurance["cause_of_loss_description"] = insurance["cause_of_loss_description"].str.lower()
insurance["cause_of_loss_description"] = insurance["cause_of_loss_description"].str.replace(
    "excess moisture/precipitation/rain", "excess-moisture", regex=True)


# 4) Define all filters based on the cleaned dataframe
# Now we can be certain we are comparing integers to integers.
year_filter = insurance["commodity_year_identifier"].between(2008, 2024)
state_filter = insurance["state_code"].eq(17) # IL
commodity_filter = insurance["commodity_code"].isin([41, 81]) # Corn, Soybeans
loss_filter = insurance["cause_of_loss_code"].isin([31, 11]) # Excess-Moisture, Drought


# 5) Combine all filters into a single master filter using the '&' operator
# This is the correct way to apply multiple conditions without index misalignment.
combined_filter = (year_filter & state_filter & commodity_filter & loss_filter)


# 6) Apply the single, combined filter to the DataFrame
insurance_final = insurance[combined_filter]


# 7) Inspect the final result
print("\n--- Final Filtered Result ---")
print("Shape:", insurance_final.shape)
print("Years:", sorted(insurance_final['commodity_year_identifier'].unique()))
print("Unique causes of loss:", insurance_final['cause_of_loss_description'].unique())

# Reset index on the final result for a clean DataFrame
insurance_final = insurance_final.reset_index(drop=True)

# Display the first few rows of the filtered data
print("\nFiltered data head:")
insurance_final.head()

--- Initial Data Inspection ---
Raw data shape: (4259695, 31)
Original data types:
commodity_year_identifier     int64
state_code                    int64
commodity_code                int64
cause_of_loss_code           object
dtype: object

--- Cleaning and Converting Data Types ---
Data types after conversion:
commodity_year_identifier    int64
state_code                   int64
commodity_code               int64
cause_of_loss_code           int64
dtype: object
Shape after cleaning: (4209876, 31)

--- Final Filtered Result ---
Shape: (62019, 31)
Years: [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Unique causes of loss: ['drought' 'excess-moisture']

Filtered data head:


,commodity_year_identifier,state_code,state_abbreviation,county_code,county_name,commodity_code,commodity_name,insurance_plan_code,insurance_abbreviation,coverage_category,...,total_premium,producer_paid_premium,subsidy,state/private_subsidy,additional_subsidy,efa_premium_discount,net_determined_quantity,indemnity_amount,loss_ratio,county_fips
0,2016,17,IL,1,Adams,41,corn,2,RP,A,...,3212.0,1445.0,1767.0,0.0,0.0,0.0,68.0,5723.0,1.78,17001
1,2016,17,IL,1,Adams,41,corn,2,RP,A,...,1928.0,1195.0,733.0,0.0,0.0,0.0,28.9,3409.0,1.77,17001
2,2016,17,IL,1,Adams,41,corn,2,RP,A,...,3517.0,2010.5,1506.5,0.0,0.0,0.0,71.2,2362.0,0.67,17001
3,2016,17,IL,1,Adams,41,corn,2,RP,A,...,688.0,426.5,261.5,0.0,0.0,0.0,5.9,338.0,0.49,17001
4,2016,17,IL,1,Adams,41,corn,2,RP,A,...,2839.0,1760.0,1079.0,0.0,0.0,0.0,56.4,14447.0,5.09,17001


In [8]:
insurance_final.to_csv('/home/emine2/CPSC-499/data/ins_corn_soy_excess_drought.csv', index=False)

In [11]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# 1) Load the datasets
# Load Illinois counties shapefile
counties = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2022/COUNTY/tl_2022_us_county.zip")
counties_il = counties[counties.STATEFP == "17"].to_crs("EPSG:4326")

# Load precipitation data (point format)
precip_df = pd.read_csv("/home/emine2/CPSC-499/data/grib_total_precipitation.csv")
precip_df['time'] = pd.to_datetime(precip_df['time'])

# Load temperature data
temp_df = pd.read_csv("/home/emine2/CPSC-499/data/grib_temperature.csv")
temp_df['time'] = pd.to_datetime(temp_df['time'])

# Inspect columns of temperature DataFrame to confirm
print("Temperature DataFrame columns:", temp_df.columns.tolist())

# 2) Spatial join: Assign CO_FIP to precipitation and temperature points
# Convert precipitation DataFrame to GeoDataFrame
geometry_precip = [Point(lon, lat) for lon, lat in zip(precip_df['longitude'], precip_df['latitude'])]
precip_gdf = gpd.GeoDataFrame(precip_df, geometry=geometry_precip, crs="EPSG:4326")

# Convert temperature DataFrame to GeoDataFrame
geometry_temp = [Point(lon, lat) for lon, lat in zip(temp_df['longitude'], temp_df['latitude'])]
temp_gdf = gpd.GeoDataFrame(temp_df, geometry=geometry_temp, crs="EPSG:4326")

# Perform spatial join to assign CO_FIP to each point
precip_with_fips = gpd.sjoin(precip_gdf, counties_il[['geometry', 'GEOID']], how="left", predicate="within")
precip_with_fips = precip_with_fips.rename(columns={'GEOID': 'CO_FIP'}).drop(columns=['geometry', 'index_right'])

temp_with_fips = gpd.sjoin(temp_gdf, counties_il[['geometry', 'GEOID']], how="left", predicate="within")
temp_with_fips = temp_with_fips.rename(columns={'GEOID': 'CO_FIP'}).drop(columns=['geometry', 'index_right'])

# 3) Average per county and time
# Average precipitation per county and time
precip_avg = precip_with_fips.groupby(['CO_FIP', 'time']).agg({
    'total_precipitation_m': 'mean',
    'latitude': 'mean',  # Optional: Keep mean lat/lon for reference
    'longitude': 'mean'
}).reset_index()

# Average temperature per county and time
temp_avg = temp_with_fips.groupby(['CO_FIP', 'time']).agg({
    'temperature_K': 'mean',  # Corrected column name
    'latitude': 'mean',  # Optional: Keep mean lat/lon for reference
    'longitude': 'mean'
}).reset_index()

# 4) Merge the averaged datasets
merged_df = pd.merge(
    precip_avg,
    temp_avg,
    on=['CO_FIP', 'time'],
    how='outer',
    suffixes=('_precip', '_temp')
)

# 5) Save the merged dataset
merged_df.to_csv("/home/emine2/CPSC-499/data/IL_county_weather_averaged.csv", index=False)
print("Merged and averaged data saved to '/home/emine2/CPSC-499/data/IL_county_weather_averaged.csv'")

Temperature DataFrame columns: ['time', 'latitude', 'longitude', 'temperature_K']
Merged and averaged data saved to '/home/emine2/CPSC-499/data/IL_county_weather_averaged.csv'


In [12]:

df = pd.read_csv("/home/emine2/CPSC-499/data/IL_county_weather_averaged.csv")
df['time'] = pd.to_datetime(df['time'])

# 2) Extract year and month from the time column
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month

# 3) Aggregate by CO_FIP, year, and month
# Sum total_precipitation_m, average temperature_K, and average lat/lon for reference
agg_df = df.groupby(['CO_FIP', 'year', 'month']).agg({
    'total_precipitation_m': 'sum',  # Sum for precipitation
    'temperature_K': 'mean',        # Mean for temperature
    'latitude_precip': 'mean',      # Mean for reference
    'longitude_precip': 'mean',
    'latitude_temp': 'mean',
    'longitude_temp': 'mean'
}).reset_index()

# 4) Create a new time column for the aggregated data (set to the first day of the month)
agg_df['time'] = pd.to_datetime(agg_df[['year', 'month']].assign(day=1))

# 5) Drop the year and month columns if not needed in the final output
agg_df = agg_df.drop(columns=['year', 'month'])

# 6) Save the aggregated dataset
agg_df.to_csv("/home/emine2/CPSC-499/data/IL_county_weather_monthly_sum.csv", index=False)
print("Aggregated data saved to '/home/emine2/CPSC-499/data/IL_county_weather_monthly_sum.csv'")

Aggregated data saved to '/home/emine2/CPSC-499/data/IL_county_weather_monthly_sum.csv'


In [ ]:
import pandas as pd

# 1) Load the datasets
# Load weather data
weather_df = pd.read_csv("/home/emine2/CPSC-499/data/IL_county_weather_monthly_sum.csv")
weather_df['time'] = pd.to_datetime(weather_df['time'])

# Load insurance data (update the path as needed)
insurance_df = pd.read_csv("/home/emine2/CPSC-499/data/ins_corn_soy_excess_drought.csv")  # Update this path

# 2) Prepare weather data for merging
# Rename CO_FIP to county_fips for consistency
weather_df = weather_df.rename(columns={'CO_FIP': 'county_fips'})

# Extract year and month from time for matching
weather_df['year'] = weather_df['time'].dt.year
weather_df['month'] = weather_df['time'].dt.month

# 3) Prepare insurance data for merging
# Ensure commodity_year_identifier and month_of_loss are integers
insurance_df['commodity_year_identifier'] = insurance_df['commodity_year_identifier'].astype(int)
insurance_df['month_of_loss'] = insurance_df['month_of_loss'].astype(int)

# 4) Merge the datasets
# Merge on county_fips, year, and month using a left join
merged_df = pd.merge(
    insurance_df,
    weather_df,
    left_on=['county_fips', 'commodity_year_identifier', 'month_of_loss'],
    right_on=['county_fips', 'year', 'month'],
    how='left'
)

# 5) Clean up the merged dataset
# Drop the temporary year and month columns from weather data
merged_df = merged_df.drop(columns=['year', 'month'])

# 6) Save the merged dataset
merged_df.to_csv("/home/emine2/CPSC-499/data/insurance_with_weather.csv", index=False)
print("Merged data saved to '/home/emine2/CPSC-499/data/insurance_with_weather.csv'")
merged_df

Merged data saved to '/home/emine2/CPSC-499/data/insurance_with_weather.csv'
